## Init e dependencies

In [8]:
! pip install langchain==0.2.7 langchain_aws langchain-community langchain_core boto3 botocore
! pip install dotenv

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [64 lines of output]
      /usr/lib/python3/dist-packages/setuptools/installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
        warnings.warn(
        error: subprocess-exited-with-error
      
        × python setup.py egg_info did not run successfully.
        │ exit code: 1
        ╰─> [16 lines of output]
            Traceback (most recent call last):
              File "<string>", line 2, in <module>
              File "<pip-setuptools-caller>", line 14, in <module>
              File "/tmp/pip-wheel-1uvasgas/distribute_dad460a4ef6f4be188c

In [6]:
! pip install pymupdf python-docx pydantic

Defaulting to user installation because normal site-packages is not writeable


In [7]:
from dotenv import load_dotenv
load_dotenv()
import os

### llm endpoints (to edit)

In [8]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [9]:
! pip install anthropic typi

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement typi (from versions: none)
ERROR: No matching distribution found for typi


In [16]:
# ! pip install  typing_extensions==4.12.2

In [31]:
import boto3
from langchain_aws import ChatBedrock
from botocore.config import Config
import urllib3
import os 
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
# from anthropic import AsyncAnthropicBedrock

bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        region_name='eu-west-1',
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
        verify=False,  # Disable SSL verification
        config=Config(
            proxies={'https': None}
        )
    )
# client = AsyncAnthropicBedrock(
#             aws_access_key=os.getenv("AWS_ACCESS_KEY_ID"),
#             aws_secret_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
#             aws_region="eu-central-1",
#         )
llm = ChatBedrock(
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        # model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
        #client=bedrock_client,
        client=bedrock_client,
        model_kwargs={
            "temperature": 0,
            "max_tokens": 2000,
        }
    )


In [17]:
import fitz  # PyMuPDF
import docx

def read_pdf(file_path):
    doc = fitz.open(file_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

def read_docx(file_path):
    doc = docx.Document(file_path)
    text = "\n".join([p.text for p in doc.paragraphs])
    return text

def read_txt(file_path):
    with open(file_path, "r") as f:
        text = f.read()
    return text



In [29]:
def riassumi(chunk):

    prompt_template = PromptTemplate(
        input_variables=["chunk"],
        template="""
            Riassumi il seguente testo mantenendo le informazioni chiave:\n\n{chunk}
        """
    )

    chain = prompt_template | llm | StrOutputParser()

    response = chain.invoke({"chunk": chunk})
    return response


In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
file_path = "docs/commedia.txt"  # Se è nella stessa cartella del notebook

text = read_txt(file_path)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100000, chunk_overlap=500)
chunks = text_splitter.split_text(text)

print(len(chunks))
    

6


In [ ]:
summaries = []
for chunk in chunks:
    response = riassumi(chunk)
    summaries.append(response)


In [35]:
from IPython.display import display, Markdown

summary = "\n".join(summaries)

display(Markdown(f"### Riassunto:\n\n{summary}"))
    

### Riassunto:

Ecco un riassunto dei principali eventi dei Canti I-XX dell'Inferno di Dante Alighieri:

Canto I: Dante si ritrova in una selva oscura e smarrita. Incontra Virgilio, che gli si offre come guida per attraversare l'Inferno e raggiungere il Paradiso.

Canto II: Beatrice, donna amata da Dante, invia Virgilio a soccorrere Dante, che è smarrito. Virgilio accetta di guidarlo.

Canto III: Dante e Virgilio giungono all'ingresso dell'Inferno, dove leggono l'iscrizione che annuncia la condanna eterna per chi entra. Entrano nell'Inferno.

Canto IV: Dante e Virgilio giungono nel Limbo, dove si trovano i grandi spiriti dell'antichità, non battezzati. Incontrano Omero, Orazio, Ovidio e Lucano.

Canto V: Dante e Virgilio giungono nel secondo cerchio, dove sono puniti i lussuriosi. Incontrano Francesca da Rimini e Paolo, che raccontano la loro storia.

Canto VI: Dante e Virgilio giungono nel terzo cerchio, dove sono puniti i golosi. Incontrano Ciacco, che profetizza le lotte future a Firenze.

Canto VII: Dante e Virgilio giungono nel quarto cerchio, dove sono puniti gli avari e i prodighi. Incontrano Pluto, il dio della ricchezza.

Canto VIII: Dante e Virgilio giungono al fiume Stige, dove sono puniti gli iracondi e gli accidiosi. Incontrano Filippo Argenti, un fiorentino violento.

Canto IX: Dante e Virgilio giungono alle mura della città di Dite, ma i demoni ne impediscono l'ingresso. Virgilio rassicura Dante.

Canto X: Dante e Virgilio entrano nella città di Dite, dove sono puniti gli eretici. Incontrano Farinata degli Uberti e altri personaggi.

Canto XI: Virgilio spiega a Dante la struttura dell'Inferno e la suddivisione dei peccati.

Canto XII: Dante e Virgilio giungono nel settimo cerchio, dove sono puniti i violenti. Incontrano il Minotauro e i centauri.

Canto XIII: Dante e Virgilio entrano nel secondo girone del settimo cerchio, dove sono puniti i suicidi. Incontrano uno spirito che si lamenta di essere stato violento contro sé stesso.

Canto XIV: Dante e Virgilio giungono al terzo girone del settimo cerchio, dove sono puniti i bestemmiatori. Incontrano Capaneo, un re che sfidò gli dei.

Canto XV: Dante incontra il suo maestro Brunetto Latini, condannato per sodomia. Brunetto gli dà consigli per il suo futuro.

Canto XVI: Dante incontra tre fiorentini illustri, condannati per violenza. Essi lo esortano a ricordare la loro fama.

Canto XVII: Dante e Virgilio incontrano Gerione, mostro simbolo della frode, che li trasporta sul fondo dell'Inferno.

Canto XVIII: Dante e Virgilio giungono nell'ottava bolgia, dove sono puniti i seduttori e i ruffiani. Incontrano Venedico Caccianemico.

Canto XIX: Dante e Virgilio giungono nella terza bolgia, dove sono puniti i simoniaci. Incontrano Papa Bonifacio VIII, condannato per simonia.

Canto XX: Dante e Virgilio giungono nella quarta bolgia, dove sono puniti gli indovini, i maghi e gli stregoni. Incontrano Tiresia e Anfiarao.
Ecco un riassunto del Canto IV del Purgatorio di Dante Alighieri:

- Dante e Virgilio riprendono il cammino verso il monte del Purgatorio. Dante è preoccupato di non essere in grado di salire, ma Virgilio lo rassicura.

- Incontrano un gruppo di anime che si muovono lentamente verso di loro. Virgilio chiede loro indicazioni per salire il monte. Le anime si fermano e una di loro, Manfredi, si rivolge a Dante.

- Manfredi racconta la sua storia: era nipote dell'imperatrice Costanza, ma morì scomunicato dalla Chiesa. Nonostante i suoi gravi peccati, la bontà divina lo ha accolto nel Purgatorio, dove sta espiando le sue colpe. 

- Manfredi chiede a Dante di andare dalla sua bella figlia e di dirle la verità sulla sua morte, smentendo le voci che circolano. Inoltre, spiega che le anime scomunicate devono attendere 30 anni prima di poter entrare nel Purgatorio.

- Manfredi invita Dante a pregare per lui, così da accorciare il suo tempo di espiazione. Poi Dante e Virgilio riprendono il cammino verso la montagna.

In sintesi, il canto presenta l'incontro con l'anima di Manfredi, che racconta la sua storia e chiede a Dante di riferire notizie alla sua famiglia, sottolineando il tema della misericordia divina e della necessità di espiare le proprie colpe.
Ecco un riassunto del Canto XXII del Purgatorio di Dante:

Il Canto inizia con Dante e Virgilio che continuano il loro cammino sul monte del Purgatorio. Incontrano un'anima che si presenta come Stazio, il poeta latino che ha scritto l'Eneide e la Tebaide. Stazio racconta di essere stato un pagano convertito al cristianesimo grazie alla lettura dell'Eneide di Virgilio. Virgilio era infatti il suo maestro e la sua guida poetica. 

Stazio spiega che il tremore del monte e le voci di lode a Dio che hanno sentito sono segno della purificazione di un'anima che sta per raggiungere il Paradiso. Egli stesso ha appena provato questa liberazione dalla pena dell'avarizia dopo essere rimasto lì per oltre cinquecento anni.

Stazio rivela inoltre di essere stato un grande poeta ai suoi tempi, famoso a Roma, ma senza ancora la vera fede cristiana. Dante si rallegra di incontrare il suo illustre predecessore e maestro, e Stazio ricambia l'affetto, essendo stato ispirato dalla lettura dell'Eneide di Virgilio.

Virgilio, vedendo l'emozione di Dante, lo invita a parlare liberamente con Stazio. Dante allora spiega a Stazio che il suo sorriso era dovuto alla meraviglia di vedere il suo maestro Virgilio, di cui Stazio era stato a sua volta discepolo. Stazio è commosso da questo legame e si alza per abbracciare Virgilio, ma Virgilio lo ferma ricordandogli che essendo ombre non possono toccarsi.

Così i tre poeti proseguono insieme il loro cammino sul monte del Purgatorio.
Ecco un riassunto dei principali contenuti del Canto VII del Paradiso di Dante Alighieri:

- Giustiniano spiega a Dante il suo ruolo di imperatore e legislatore, e come abbia contribuito a diffondere la vera fede cristiana.

- Giustiniano racconta le imprese dell'aquila romana, simbolo dell'Impero, e come essa abbia dominato il mondo per secoli, fino ai tempi di Cesare.

- Giustiniano sottolinea la grandezza e la sacralità dell'Impero romano, che ha svolto un ruolo provvidenziale nella storia dell'umanità.

- Egli critica coloro che si oppongono all'aquila imperiale, come i Ghibellini, e invita i Guelfi a non combatterla, perché ciò andrebbe contro la volontà di Dio.

- Giustiniano parla poi di Romeo, un umile e giusto uomo che servì fedelmente il conte di Provenza, ma fu poi ingiustamente accusato e costretto a vivere in povertà.

- Questo episodio dimostra come spesso il bene fatto agli altri non venga riconosciuto e apprezzato, ma anzi sia motivo di invidia e di danno per chi lo compie.

In sintesi, il canto approfondisce il tema dell'Impero romano come strumento della Provvidenza divina, e affronta il problema della giustizia umana, spesso distorta dalle passioni e dagli interessi personali.
Ecco un riassunto delle informazioni chiave contenute nei canti VII-XXV del Paradiso:

Canto VII: Beatrice spiega a Dante il mistero della redenzione umana, spiegando come il peccato di Adamo fu giustamente punito sulla croce di Cristo.

Canto VIII: Dante incontra l'anima di Cunizza da Romano, che gli rivela il futuro di alcune città italiane. Poi incontra l'anima di Folco di Marsiglia.

Canto IX: Dante incontra l'anima di Cacciaguida, suo antenato, che gli racconta della Firenze del suo tempo, più semplice e virtuosa.

Canto X: Dante contempla la bellezza della Trinità e la gerarchia degli spiriti beati. Cacciaguida gli spiega la predestinazione divina.

Canto XI: Cacciaguida parla di San Francesco d'Assisi e del declino dell'ordine francescano.

Canto XII: Dante incontra le anime di altri santi, come San Domenico e San Tommaso d'Aquino, che gli spiegano la gloria del paradiso.

Canto XIII-XIV: Dante riceve ulteriori spiegazioni sulla predestinazione e sulla gloria del paradiso.

Canto XV-XVI: Cacciaguida continua a raccontare a Dante della Firenze del suo tempo, più semplice e virtuosa rispetto a quella corrotta del presente.

Canto XVII: Cacciaguida profetizza a Dante l'esilio e le sofferenze che lo attendono.

Canto XVIII-XIX: Dante riceve ulteriori spiegazioni sulla fede e sulla giustizia divina da parte di un'altra anima beata.

Canto XX-XXI: Dante contempla la bellezza del paradiso e la gloria di Dio.

Canto XXII-XXIII: Dante sale sempre più in alto nel paradiso, ammirando la bellezza e la luce divina.

Canto XXIV: Dante viene interrogato sulla fede da un'anima beata, dimostrando la sua profonda convinzione.

Canto XXV: Dante viene benedetto dall'apostolo San Giacomo per la sua fede.
Ecco un riassunto delle informazioni chiave contenute nei tre canti del Paradiso:

Canto XXV:
- Il poeta desidera tornare a scrivere poesia dopo aver affrontato il "bello ovile" (la Chiesa) 
- Appare una luce che rappresenta San Giacomo, patrono di Compostela
- Il poeta è interrogato sulla speranza, che definisce come "attender certo della gloria futura"

Canto XXVI: 
- Il poeta spiega come l'amore per Dio sia fondato sulla conoscenza della sua bontà e perfezione
- Vengono descritti gli ordini angelici, dalla più alta gerarchia (Serafini e Cherubini) a quella più bassa
- Si chiarisce che i bambini morti senza battesimo sono comunque accolti in Paradiso

Canto XXVII:
- Il poeta descrive la visione della Trinità, rappresentata da tre cerchi di luce
- Beatrice spiega come la Chiesa sia stata corrotta dalla cupidigia e dal potere temporale
- Si profetizza l'avvento di un imperatore (Enrico VII) che riporterà l'ordine nella Chiesa

In sintesi, i canti trattano della poesia, della teologia della speranza e della Trinità, nonché della critica alla corruzione della Chiesa.

In [42]:

def cerca(chunk, query):

    prompt_template = PromptTemplate(
        input_variables=["chunk", "query"],
        template="""
            Rispondi alla richiesta dell'utente in base al chunk in input:\n\n
            <query>
            {query}
            </query>
            
            <testo>
            {chunk}
            </testo>
            
            ISTRUZIONI: 
            se trovi una corrispondenza semantica alla richiesta dell'utente, scrivi la risposta. altrimenti non rispondere
            ritorna anche il paragrafo dove hai trovato la risposta
        """
    )

    chain = prompt_template | llm | StrOutputParser()

    response = chain.invoke({"chunk": chunk, "query": query})
    return response

for chunk in chunks:
    response = cerca(chunk, "che ha fatto il conte ugolino?")
    print(response)
    

Purtroppo non ho trovato una corrispondenza semantica diretta alla richiesta dell'utente "che ha fatto il conte ugolino?" all'interno del testo fornito. Il testo è tratto dalla Divina Commedia di Dante Alighieri e non contiene informazioni sul Conte Ugolino.
Sì, ho trovato una corrispondenza semantica alla richiesta dell'utente. Ecco la risposta e il paragrafo corrispondente:

Il conte Ugolino ha fatto il seguente:
- È stato rinchiuso nella torre della fame a Pisa insieme ai suoi figli e nipoti, dove sono morti di fame. 
- È stato tradito dall'arcivescovo Ruggieri, che lo ha fatto imprigionare e morire di fame.

Questo è descritto nel Canto XXXIII del Purgatorio:

"Io fui d'Arezzo, e Albero da Siena",
rispuose l'un, "mi fé mettere al foco;
ma quel per ch'io mori' qui non mi mena.
Vero è ch'i' dissi lui, parlando a gioco:
"I' mi saprei levar per l'aere a volo";
e quei, ch'avea vaghezza e senno poco,
volle ch'i' li mostrassi l'arte; e solo
perch'io nol feci Dedalo, mi fece
ardere a tal c

Leggere documento in input
summarization semplice con documento piccolo (fanno loro)

Si tenta di fare summarization di doc troppo grande (es. promessi sposi) - schiatta

Splitter

Demo: riassunto e riassunto dei riassunti

per ogni chunk:
- eseguire ricerca, ad esempio "chi è l'amata di Dante"
- l'output della ricerca va in una struttura: (trovato, risposta, paragrafo_corrispondenza)
- se trovata corrispondenza, stampa

risposta: beatrice è l'amata di Dante
contesto: versi in cui si cita beatrice
trovato: true

PRO e CONTRO:
- semplice

- costi e prestazioni

alternativa: RAG